# pandas: lendo da internet

### Essa é uma das formas de importar arquivos da internet, direto do pandas. Mas nem sempre você vai conseguir abrir assim direto.


### dados em uma requisição que precisa ser tratada

Histórico de da produçaõ de café

In [289]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io

link = 'https://portalweb.cooxupe.com.br:9080/portal/precohistoricocafe.jsp'
header = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:122.0) Gecko/20100101 Firefox/122.0"}

requisicao = requests.get(link, headers=header)

site = BeautifulSoup(requisicao.text, "html.parser")

table = site.find("table", attrs={"class":"display"})

# O primeiro tr contém o cabecalho (meses).

mes = [th.get_text() for th in table.find("tr").find_all("th")]
mes = mes[1:]

dados = ([th.get_text().replace('\r\n                    ','').replace(' ','') for th in table.find_all("td")])

ano = []
valor = []
for i, item in enumerate(dados):
    if len(dados[i]) == 4 and '.' not in dados[i] :
        ano.append(dados[i])
    else:
        valor.append(dados[i])

In [304]:
# Criando uma lista de dicionários com ano, mês e valor

dados = []

for i in range(len(valor)):
    ano_index = i // 12
    mes_index = i % 12
    dados.append({'ano': ano[ano_index], 'mes': mes[mes_index], 'valor': float(valor[i])})

# Criando um DataFrame a partir da lista de dicionários

df = pd.DataFrame(dados)
df = df.pivot(index='ano', columns='mes', values='valor')
df = df.reindex(columns=mes)
df = df.sort_index(ascending = False)
df = df.reset_index()

# Exibindo o DataFrame

df.to_excel("HistoricoProducaoCafe.xlsx", index=False)